In [24]:
import os
import sys
import pandas as pd

notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
# Add the parent directory to the Python path
sys.path.append(parent_dir)

from folktables import ACSDataSource, ACSEmployment
from src.train_fair import FairLogisticRegression
from src.data_utils import joint_distribution
from src.analyze_metric_sensitivity import analyze_metric_bias_sensitivity
import numpy as np

In [14]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["AL"], download=True)
features, label, group = ACSEmployment.df_to_numpy(acs_data)

In [15]:
prop = int(0.8 * label.shape[0])
perm = np.random.choice(label.shape[0], size=(label.shape[0],), replace=False)

X_train, y_train, A_train = features[:-prop], label[:-prop], group[perm][:-prop]
X_test, y_test, A_test = features[perm][-prop:], label[perm][-prop:], group[perm][-prop:]

In [16]:
FairLR = FairLogisticRegression(fairness_constraint="FPR",grid_size=20)

In [17]:
FairLR.fit(X_train,y_train, A_train)

The grid has 7 dimensions. It is not recommended to use more than 4, otherwise a prohibitively large grid size is required to explore the space thoroughly. For such cases consider using ExponentiatedGradient from the fairlearn.reductions module.
Generating a grid with 20 grid points. It is recommended to use at least 128 grid points. Please consider increasing grid_size.


/Users/jake/miniconda3/envs/test_fair/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/jake/miniconda3/envs/test_fair/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

In [20]:
test_df = {"Y": y_test.astype(int),"P":FairLR.predict(X_test), "A":A_test.astype(int)}
test_df = pd.DataFrame(test_df)

joint_dist = joint_distribution(test_df,variables=list(test_df.columns))

In [31]:
analyze_metric_bias_sensitivity(joint_dist,metric="FNR",bias="selection",sensitivity_parameter_values=0.03)

ValueError: Cannot load a SolverResults object with bad status: error